In [1]:
import cv2
import mediapipe as mp
import math
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        #print(results.multi_hand_landmarks) #prints if there is solution or not, so if a hand is not seen it will print none if a hand is seen it gives the landmarks coordinats
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extract finger landmark positions
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                # Calculate the Euclidean distance between the index and thumb tips
                thumb_x, thumb_y = int(thumb_tip.x * image.shape[1]), int(thumb_tip.y * image.shape[0])
                index_x, index_y = int(index_tip.x * image.shape[1]), int(index_tip.y * image.shape[0])
                line_length = int(math.sqrt((thumb_x - index_x)**2 + (thumb_y - index_y)**2))
                # for clarfication  image.shape[1] and image.shape[0] is the width and height of 
                # the image then simple multiplication is done to get the coordinates of the landmarks and then simple vector to gethe length of the line

                # Draw a line between the index and thumb tips
                cv2.line(image, (thumb_x, thumb_y), (index_x, index_y), (0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

                # Print the length of the line
                cv2.putText(image, f"Line length: {line_length}px", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('MediaPipe Hands', image)



        if cv2.waitKey(5) & 0xFF == 27:
            break

if cv2.waitKey(1)==27:
    cv2.destroyAllWindows()
    cap.release()
    


KeyboardInterrupt: 